In [1]:
#!pip install anthropic

In [2]:
import os
from anthropic import Anthropic

In [3]:
import pandas as pd

In [45]:
from tqdm import tqdm

In [26]:
from IPython.display import display, Markdown

In [57]:
import re

In [82]:
import xml.etree.ElementTree as ET

In [116]:
#os.environ['ANTHROPIC_API_KEY'] = ''

In [5]:

client = Anthropic(
    # This is the default and can be omitted
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)


In [9]:
all_demos = pd.read_parquet('../data/latest-demos/all_demo_python.parquet')

In [10]:
this_demo = all_demos.iloc[62]

In [19]:
its = this_demo['input type(s)']

In [20]:
ots = this_demo['output type(s)']

In [63]:
# Prompt template
prompt_template = """
You will be acting as a dataset generator to help develop a fine-tuning dataset for training an AI coding assistant for the Gradio Python framework. 
You will return a response as though you are writting the initial prompt that generated the SOLUTION_EXAMPLE

Here is an example solution in Python code:

<solution_example>
{{SOLUTION_EXAMPLE}}
</solution_example>

And here is some information about the solution:

<solution_information>
{{SOLUTION_INFORMATION}}
</solution_information>

Imagine that you were the human who first thought of the idea to develop that Gradio solution. Firstly give a brief overview of the non gradio components of the code amd summarize or generalize 
them such as "I have ... and with my current model/code I can produce an output ..." then go into more detail on what you would like gradio to do, what features Gradio you want to use which components need to be interactice etc etc.

For example, the prompt might look something like:
<prompt>
I have a machine learning model that can take in text and classify the sentiment as positive or negative. With my current code, I can run the model and get the predicted sentiment label as output. 

I would like to create a simple Gradio web interface for my model with the following:
- A text input box for the user to enter the text to classify 
- A submit button to run the classification
- Display the output sentiment label after submitting
- A clear button to reset the interface 
</prompt>

Please return only the prompt that could have generated the given solution.
"""



In [96]:
responses = []

In [97]:
for i in tqdm(range(len(all_demos))):
    this_demo = all_demos.iloc[i]
    prompt = prompt_template.replace("{{SOLUTION_EXAMPLE}}", f"```python\n{this_demo.python_str}\n```").replace("{{SOLUTION_INFORMATION}}",f'this solution has been categorised as {this_demo.level_0} and contains {its} input types and the solution outputs {ots}  types ')
    message = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1000,
        temperature=0.1,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    )
    # Append the response to the list
    responses.append(message.content[0].text)
    counter_incase_crash+=1


100%|███████████████████████████████████████████████████████████████████████████████████| 67/67 [14:41<00:00, 13.16s/it]


In [98]:
# Print the responses
responses_clean=[]
for response in responses:
    root = ET.fromstring(f"<root>{response}</root>")
    result_element = root.find('prompt')
    result_content = result_element.text
    responses_clean.append(result_content)

In [100]:
responses_df = pd.Series(responses_clean).rename('question')

In [105]:
as_answers = all_demos.python_str.apply(lambda x: f'here is the gradio solution for your use case: ```python\n{x}\n```').rename('answer')

In [108]:
qa_out = pd.concat([responses_df,as_answers],axis=1).astype(str)
qa_out.to_parquet(os.getcwd() + f'/../datasets/raw/opus_demos_qas.parquet')

In [114]:
output_json_filepath = os.getcwd() + f'/../datasets/qa_pairs/opus_demos_qas.jsonl'

In [115]:
qa_out.to_json(output_json_filepath,orient='records',lines=True)